## Predict Future Sales

Final project for "How to win a data science competition" Coursera course

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

**Farrukh Bulbulov**